In [47]:
import pandas as pd
import numpy as np
import matplotlib as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from datetime import datetime
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,classification_report,mean_absolute_error,r2_score
from sklearn.linear_model import LinearRegression

from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import Adam

le=LabelEncoder()
lr=LinearRegression()

Using TensorFlow backend.


In [2]:
traffic=pd.read_csv("Train.csv")
traffic

,date_time,is_holiday,air_pollution_index,humidity,wind_speed,wind_direction,visibility_in_miles,dew_point,temperature,rain_p_h,snow_p_h,clouds_all,weather_type,weather_description,traffic_volume
0,02-10-12 9:00,None,121,89,2,329,1,1,288.28,0.0,0.0,40,Clouds,scattered clouds,5545
1,02-10-12 10:00,None,178,67,3,330,1,1,289.36,0.0,0.0,75,Clouds,broken clouds,4516
2,02-10-12 11:00,None,113,66,3,329,2,2,289.58,0.0,0.0,90,Clouds,overcast clouds,4767
3,02-10-12 12:00,None,20,66,3,329,5,5,290.13,0.0,0.0,90,Clouds,overcast clouds,5026
4,02-10-12 13:00,None,281,65,3,329,7,7,291.14,0.0,0.0,75,Clouds,broken clouds,4918
5,02-10-12 14:00,None,23,65,3,328,6,6,291.72,0.0,0.0,1,Clear,sky is clear,5181
6,02-10-12 15:00,None,184,64,3,328,7,7,293.17,0.0,0.0,1,Clear,sky is clear,5584
7,02-10-12 16:00,None,167,64,3,327,7,7,293.86,0.0,0.0,1,Clear,sky is clear,6015
8,02-10-12 17:00,None,119,63,3,327,6,6,294.14,0.0,0.0,20,Clouds,few clouds,5791
9,02-10-12 18:00,None,161,63,3,326,3,3,293.10,0.0,0.0,20,Clouds,few clouds,4770


In [3]:
print(traffic.shape)

(33750, 15)


In [4]:
traffic["is_holiday"].value_counts()

None                         33707
Thanksgiving Day                 5
Christmas Day                    5
New Years Day                    5
Washingtons Birthday             4
Columbus Day                     4
Labor Day                        4
Veterans Day                     4
Memorial Day                     3
Martin Luther King Jr Day        3
Independence Day                 3
State Fair                       3
Name: is_holiday, dtype: int64

In [5]:
#label encoding weather_type to numerical
traffic.weather_type=le.fit_transform(traffic.weather_type)

In [6]:
traffic.head()

,date_time,is_holiday,air_pollution_index,humidity,wind_speed,wind_direction,visibility_in_miles,dew_point,temperature,rain_p_h,snow_p_h,clouds_all,weather_type,weather_description,traffic_volume
0,02-10-12 9:00,None,121,89,2,329,1,1,288.28,0.0,0.0,40,1,scattered clouds,5545
1,02-10-12 10:00,None,178,67,3,330,1,1,289.36,0.0,0.0,75,1,broken clouds,4516
2,02-10-12 11:00,None,113,66,3,329,2,2,289.58,0.0,0.0,90,1,overcast clouds,4767
3,02-10-12 12:00,None,20,66,3,329,5,5,290.13,0.0,0.0,90,1,overcast clouds,5026
4,02-10-12 13:00,None,281,65,3,329,7,7,291.14,0.0,0.0,75,1,broken clouds,4918


In [7]:
list(le.classes_)

['Clear',
 'Clouds',
 'Drizzle',
 'Fog',
 'Haze',
 'Mist',
 'Rain',
 'Smoke',
 'Snow',
 'Squall',
 'Thunderstorm']

In [8]:
#converting is_holiday to numerical; 1 if holiday and 0 otherwise
traffic.loc[traffic['is_holiday']!='None','is_holiday']=1
traffic.loc[traffic['is_holiday']=='None','is_holiday']=0
traffic

,date_time,is_holiday,air_pollution_index,humidity,wind_speed,wind_direction,visibility_in_miles,dew_point,temperature,rain_p_h,snow_p_h,clouds_all,weather_type,weather_description,traffic_volume
0,02-10-12 9:00,0,121,89,2,329,1,1,288.28,0.0,0.0,40,1,scattered clouds,5545
1,02-10-12 10:00,0,178,67,3,330,1,1,289.36,0.0,0.0,75,1,broken clouds,4516
2,02-10-12 11:00,0,113,66,3,329,2,2,289.58,0.0,0.0,90,1,overcast clouds,4767
3,02-10-12 12:00,0,20,66,3,329,5,5,290.13,0.0,0.0,90,1,overcast clouds,5026
4,02-10-12 13:00,0,281,65,3,329,7,7,291.14,0.0,0.0,75,1,broken clouds,4918
5,02-10-12 14:00,0,23,65,3,328,6,6,291.72,0.0,0.0,1,0,sky is clear,5181
6,02-10-12 15:00,0,184,64,3,328,7,7,293.17,0.0,0.0,1,0,sky is clear,5584
7,02-10-12 16:00,0,167,64,3,327,7,7,293.86,0.0,0.0,1,0,sky is clear,6015
8,02-10-12 17:00,0,119,63,3,327,6,6,294.14,0.0,0.0,20,1,few clouds,5791
9,02-10-12 18:00,0,161,63,3,326,3,3,293.10,0.0,0.0,20,1,few clouds,4770


In [9]:
traffic['is_holiday'].value_counts()

0    33707
1       43
Name: is_holiday, dtype: int64

In [10]:
traffic['date_time'].describe()

count              33750
unique             28589
top       18-04-13 22:00
freq                   6
Name: date_time, dtype: object

In [11]:
#splitting date_time into date and time
traffic[['date','time']]=traffic.date_time.str.split(expand=True)
traffic.head()

,date_time,is_holiday,air_pollution_index,humidity,wind_speed,wind_direction,visibility_in_miles,dew_point,temperature,rain_p_h,snow_p_h,clouds_all,weather_type,weather_description,traffic_volume,date,time
0,02-10-12 9:00,0,121,89,2,329,1,1,288.28,0.0,0.0,40,1,scattered clouds,5545,02-10-12,9:00
1,02-10-12 10:00,0,178,67,3,330,1,1,289.36,0.0,0.0,75,1,broken clouds,4516,02-10-12,10:00
2,02-10-12 11:00,0,113,66,3,329,2,2,289.58,0.0,0.0,90,1,overcast clouds,4767,02-10-12,11:00
3,02-10-12 12:00,0,20,66,3,329,5,5,290.13,0.0,0.0,90,1,overcast clouds,5026,02-10-12,12:00
4,02-10-12 13:00,0,281,65,3,329,7,7,291.14,0.0,0.0,75,1,broken clouds,4918,02-10-12,13:00


In [12]:
#rearranging date and time to dataframe beginning
col=traffic.columns.tolist()
col=col[-1:]+col[:-1]
col=col[-1:]+col[:-1]
#storing column sequence in col
col

['date',
 'time',
 'date_time',
 'is_holiday',
 'air_pollution_index',
 'humidity',
 'wind_speed',
 'wind_direction',
 'visibility_in_miles',
 'dew_point',
 'temperature',
 'rain_p_h',
 'snow_p_h',
 'clouds_all',
 'weather_type',
 'weather_description',
 'traffic_volume']

In [13]:
#using stored column sequence to rearrange dataframe
traffic=traffic[col]
traffic=traffic.drop('date_time',axis=1)
traffic.head()

,date,time,is_holiday,air_pollution_index,humidity,wind_speed,wind_direction,visibility_in_miles,dew_point,temperature,rain_p_h,snow_p_h,clouds_all,weather_type,weather_description,traffic_volume
0,02-10-12,9:00,0,121,89,2,329,1,1,288.28,0.0,0.0,40,1,scattered clouds,5545
1,02-10-12,10:00,0,178,67,3,330,1,1,289.36,0.0,0.0,75,1,broken clouds,4516
2,02-10-12,11:00,0,113,66,3,329,2,2,289.58,0.0,0.0,90,1,overcast clouds,4767
3,02-10-12,12:00,0,20,66,3,329,5,5,290.13,0.0,0.0,90,1,overcast clouds,5026
4,02-10-12,13:00,0,281,65,3,329,7,7,291.14,0.0,0.0,75,1,broken clouds,4918


In [14]:
#converting date to datetime type to get days from it further on
traffic['date']= pd.to_datetime(traffic['date'])

In [15]:
traffic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33750 entries, 0 to 33749
Data columns (total 16 columns):
date                   33750 non-null datetime64[ns]
time                   33750 non-null object
is_holiday             33750 non-null int64
air_pollution_index    33750 non-null int64
humidity               33750 non-null int64
wind_speed             33750 non-null int64
wind_direction         33750 non-null int64
visibility_in_miles    33750 non-null int64
dew_point              33750 non-null int64
temperature            33750 non-null float64
rain_p_h               33750 non-null float64
snow_p_h               33750 non-null float64
clouds_all             33750 non-null int64
weather_type           33750 non-null int32
weather_description    33750 non-null object
traffic_volume         33750 non-null int64
dtypes: datetime64[ns](1), float64(3), int32(1), int64(9), object(2)
memory usage: 4.0+ MB


In [16]:
traffic1=traffic
traffic1.head()

,date,time,is_holiday,air_pollution_index,humidity,wind_speed,wind_direction,visibility_in_miles,dew_point,temperature,rain_p_h,snow_p_h,clouds_all,weather_type,weather_description,traffic_volume
0,2012-02-10,9:00,0,121,89,2,329,1,1,288.28,0.0,0.0,40,1,scattered clouds,5545
1,2012-02-10,10:00,0,178,67,3,330,1,1,289.36,0.0,0.0,75,1,broken clouds,4516
2,2012-02-10,11:00,0,113,66,3,329,2,2,289.58,0.0,0.0,90,1,overcast clouds,4767
3,2012-02-10,12:00,0,20,66,3,329,5,5,290.13,0.0,0.0,90,1,overcast clouds,5026
4,2012-02-10,13:00,0,281,65,3,329,7,7,291.14,0.0,0.0,75,1,broken clouds,4918


In [17]:
#using only days from the dates; converting days to numerical equivalents
traffic1['date']=traffic1['date'].dt.weekday
traffic1['date']=traffic1['date'].astype(str)

In [18]:
traffic1.head()

,date,time,is_holiday,air_pollution_index,humidity,wind_speed,wind_direction,visibility_in_miles,dew_point,temperature,rain_p_h,snow_p_h,clouds_all,weather_type,weather_description,traffic_volume
0,4,9:00,0,121,89,2,329,1,1,288.28,0.0,0.0,40,1,scattered clouds,5545
1,4,10:00,0,178,67,3,330,1,1,289.36,0.0,0.0,75,1,broken clouds,4516
2,4,11:00,0,113,66,3,329,2,2,289.58,0.0,0.0,90,1,overcast clouds,4767
3,4,12:00,0,20,66,3,329,5,5,290.13,0.0,0.0,90,1,overcast clouds,5026
4,4,13:00,0,281,65,3,329,7,7,291.14,0.0,0.0,75,1,broken clouds,4918


In [19]:
#dropping the :00 from time since redundant and inhibits operations
traffic1['time']=traffic1['time'].str.replace(':00','')
traffic1.head()

,date,time,is_holiday,air_pollution_index,humidity,wind_speed,wind_direction,visibility_in_miles,dew_point,temperature,rain_p_h,snow_p_h,clouds_all,weather_type,weather_description,traffic_volume
0,4,9,0,121,89,2,329,1,1,288.28,0.0,0.0,40,1,scattered clouds,5545
1,4,10,0,178,67,3,330,1,1,289.36,0.0,0.0,75,1,broken clouds,4516
2,4,11,0,113,66,3,329,2,2,289.58,0.0,0.0,90,1,overcast clouds,4767
3,4,12,0,20,66,3,329,5,5,290.13,0.0,0.0,90,1,overcast clouds,5026
4,4,13,0,281,65,3,329,7,7,291.14,0.0,0.0,75,1,broken clouds,4918


In [20]:
traffic1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33750 entries, 0 to 33749
Data columns (total 16 columns):
date                   33750 non-null object
time                   33750 non-null object
is_holiday             33750 non-null int64
air_pollution_index    33750 non-null int64
humidity               33750 non-null int64
wind_speed             33750 non-null int64
wind_direction         33750 non-null int64
visibility_in_miles    33750 non-null int64
dew_point              33750 non-null int64
temperature            33750 non-null float64
rain_p_h               33750 non-null float64
snow_p_h               33750 non-null float64
clouds_all             33750 non-null int64
weather_type           33750 non-null int32
weather_description    33750 non-null object
traffic_volume         33750 non-null int64
dtypes: float64(3), int32(1), int64(9), object(3)
memory usage: 4.0+ MB


In [21]:
#converting all sundays to holidays in is_holiday
traffic1.loc[traffic1['date']=='6','is_holiday']=1

In [26]:
#transferring dataframe to traffic2 for dropping weather_description; traffic1 to be worked on later
traffic2=traffic1
traffic2=traffic2.drop(['weather_description'],axis=1)
traffic2.head()

,date,time,is_holiday,air_pollution_index,humidity,wind_speed,wind_direction,visibility_in_miles,dew_point,temperature,rain_p_h,snow_p_h,clouds_all,weather_type,traffic_volume
0,4,9,0,121,89,2,329,1,1,288.28,0.0,0.0,40,1,5545
1,4,10,0,178,67,3,330,1,1,289.36,0.0,0.0,75,1,4516
2,4,11,0,113,66,3,329,2,2,289.58,0.0,0.0,90,1,4767
3,4,12,0,20,66,3,329,5,5,290.13,0.0,0.0,90,1,5026
4,4,13,0,281,65,3,329,7,7,291.14,0.0,0.0,75,1,4918


In [76]:
x_train,x_test,y_train,y_test=train_test_split(traffic2.drop('traffic_volume',axis=1),traffic2['traffic_volume'],test_size=0.2,random_state=5)

In [77]:
lr.fit(x_train,y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

In [78]:
prd=lr.predict(x_test)

In [79]:
lr.score(x_test,y_test)

0.15082154889992094

In [81]:
r_sq=r2_score(y_test,prd)
r_sq

0.15082154889992094

A 2% to 15% jump is good, but far from acceptable. I'm gonna work on the weather decsription column instead of dropping it, as well as simplifying most other data. Do discuss any ideas or plans-of-action on the group.

### Here goes CNN... (this is the part where you stop reading)

(Intentionally didn't delete this cell before sending, so y'all can apply ideas which may increase accuracy here.)

In [96]:
adam=Adam(lr=0.001, beta_1=0.999, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
model = Sequential()
model.add(Dense(64, input_dim=14, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(16, activation='softmax'))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='mean_squared_error',
              optimizer=adam,
              metrics=['accuracy'])

model.fit(x_train, y_train,
          epochs=10,
          batch_size=1000)
score = model.evaluate(x_test,y_test,batch_size=1000)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Epoch 1/10
27000/27000 [==============================] - 2s 60us/step - loss: 14409947.0370 - acc: 1.4815e-04
Epoch 2/10
27000/27000 [==============================] - 0s 13us/step - loss: 14409598.1481 - acc: 1.4815e-04
Epoch 3/10
27000/27000 [==============================] - 0s 14us/step - loss: 14409451.6667 - acc: 1.4815e-04
Epoch 4/10
27000/27000 [==============================] - 0s 14us/step - loss: 14409385.3704 - acc: 1.4815e-04
Epoch 5/10
27000/27000 [==============================] - 0s 14us/step - loss: 14409314.7407 - acc: 1.4815e-04
Epoch 6/10
27000/27000 [==============================] - 0s 17us/step - loss: 14409245.2222 - acc: 1.4815e-04
Epoch 7/10
27000/27000 [==============================] - 0s 15us/step - loss: 14409174.5556 - acc: 1.4815e-04
Epoch 8/10
27000/27000 [==============================] - 0s 14us/step - loss: 14409104.4444 - acc: 1.4815e-04
Epoch 9/10
27000/27000 [==============================] - 0s 16us/step - loss: 14409037.3333 - acc: 1.4815e-04
E